# Analyzing Assessed Value of the Downtown Mall

## Goals: 

    *Provide descriptive statistics of assessed value over time for the City of Charlottesville's downtown mall

    *Map and chart assessed values over time

## Step 1: Aquire Data

### Assessment Values
    Charlottesville's Open Data Portal : http://opendata.charlottesville.org/

    Real Estate (All Assessments) Dataset : http://opendata.charlottesville.org/datasets/real-estate-all-assessments

        * On the Real Estate dataset page, in the upper right corner of the window under the map click the APIs             drop down
        * Copy the GeoJSON link
        * Use the GeoJSON link to pull data directly from the Open Data portal using the code below

    Parcel Area Data : http://opendata.charlottesville.org/datasets/parcel-boundary-area

### List of properties to use in analysis

Charlottesville GIS Viewer: https://gisweb.charlottesville.org/GisViewer/

    * Under Map option, turn on 'Parcels & Buildings' > 'Parcels'. Turn everything else off.

    * Zoom to area of interest on map

    * Under 'Tools' select 'Identify'

    * In 'Identify' toolbar select 'Custom Shape' and under 'Layer' select 'Parcels'

    * Using mouse on map, click boundary around area of interest

    * A list will appear in the left panel of the web page

    * In the panel click 'Tools' > 'Export All to Excel'

    * A window named 'Export Results' will open when your download is ready.

    * Click 'View Export' and save file to your project directory

<img src="https://github.com/strmwtr/downtown_assessments/blob/master/img/getting_pin_list.png?raw=true">

# Step 2: Prepare Data

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import requests
import folium

%matplotlib inline

## Import .xls retrieved from the GIS Viewer into pandas

In [2]:
#Path to the .xls retrieve from the GIS Viewer
f = r'./data/pin_exp.xls'

#Create a dataframe that reads the .xls file
df = pd.read_excel(f)

#### Remove all MULTIPIN parcels from df

In [3]:
#Identify all rows in df where MULTIPIN column is not equal to 1
not_multipin = df['MULTIPIN'] != 1
#Create a new dataframe that only contains the rows identified in not_multipin
df = df[not_multipin]
df.head()

FullAddress  OBJECTID        PIN   GPIN ParcelNumber  \
0                0 3RD ST SE   24846953  280036300   7309    280036300   
1              0 E MARKET ST   24848655  330245100  14744    330245100   
2  0 W MARKET ST & 2ND ST NW   24841086  330262000   6656    330262000   
3              100 5TH ST SE   24845005  530065300   7426    530065300   
4              100 E MAIN ST   24839773  280020000   7021    280020000   

                                     OwnerName  CurrentAssessedValue  \
0                            LITTLE MOOSE, LLC                204100   
1           FIRST AND MAIN CHARLOTTESVILLE LLC               1122200   
2  SPENCER, HAWES, ETAL, TR PROTICO PROP LD TR                418400   
3           MAIN, RALPH TR OF BLACK DUCK LD TR                664000   
4               ONE HUNDRED EAST MAIN LTD PART               1904200   

  CurrentTaxYear CurrentAssessedValueWithLabel  \
0    2019 Value:          2019 Value:  204,100   
1    2019 Value:        2019 Value:  1,122,200   
2    2019 Value:          2019 Value:  418,400   
3    2019 Value:          2019 Value:  664,000   
4    2019 Value:        2019 Value:  1,904,200   

                                         PicturePath  ... MULTIPIN  \
0  http://realestate.charlottesville.org/IMAGES\P...  ...      NaN   
1                                                NaN  ...      NaN   
2  http://realestate.charlottesville.org/IMAGES\P...  ...      NaN   
3  http://realestate.charlottesville.org/IMAGES\P...  ...      NaN   
4  http://realestate.charlottesville.org/IMAGES\P...  ...      NaN   

         OwnerAddress      OwnerCityState  OwnerZipCode SHAPE.STArea()  \
0        P O BOX 4226  CHARLOTTESVILLE VA         22905       1110.750   
1  224 14TH STREET NW  CHARLOTTESVILLE VA         22903      11316.250   
2       700 E HIGH ST  CHARLOTTESVILLE VA         22902       3802.250   
3        P O BOX 2378  CHARLOTTESVILLE VA         22902        787.375   
4        MSC BOX 5186  CHARLOTTESVILLE VA         22905       5102.750   

  SHAPE.STLength() cvGIS.CITY.parcel_area.CreatedBy  \
0       178.976929                              NaN   
1       425.844859                              NaN   
2       254.696684                              NaN   
3       118.075826                              NaN   
4       418.086500                              NaN   

   cvGIS.CITY.parcel_area.CreatedDate  cvGIS.CITY.parcel_area.ModifiedBy  \
0                                 NaN                                NaN   
1                                 NaN                                NaN   
2                                 NaN                                NaN   
3                                 NaN                                NaN   
4                                 NaN                                NaN   

  cvGIS.CITY.parcel_area.ModifiedDate  
0                                 NaN  
1                                 NaN  
2                                 NaN  
3                                 NaN  
4                                 NaN  

[5 rows x 23 columns]

<b>Use the dataset that removes the Omni, Fed, and Charlottesville parcels that skew our results and do not represent the target audience. Uncomment/comment if you want to use it

In [4]:
#Path to the .xls retrieve from the GIS Viewer
#f = r'./data/mall_parcels_no_ends.xls'
#Create a dataframe that reads the .xls file
#df = pd.read_excel(f)
#Identify all rows in df where MULTIPIN column is not equal to 1
#not_multipin = df['MULTIPIN'] != 1
#Create a new dataframe that only contains the rows identified in not_multipin
#df = df[not_multipin]

## Access json file for parcel areas

In [5]:
formatted_gpins = [str(x) for x in df['GPIN'].unique()]
formatted_gpins = formatted_gpins
formatted_gpins = ','.join(formatted_gpins)

#Query url
parcel_url = f'https://opendata.arcgis.com/datasets/320d465ddf0d498796da6491e21f6dde_43.geojson?where=GPIN%20in%20({formatted_gpins})'
parcel_request = requests.get(parcel_url)
parcel_json = parcel_request.json()

## Preparing annual assessment data

In [6]:
formatted_pins = [f'%27{x}%27' for x in df['PIN'].unique()]
formatted_pins_1 = formatted_pins
formatted_pins_1 = ','.join(formatted_pins_1)

assessment_url = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins_1})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear&outSR=4326&f=json"

assessment_request = requests.get(assessment_url)

d1 = assessment_request.json()

## Create data series based on features and combine data frames into a single df

In [7]:
df1 = pd.DataFrame(d1['features'])

## Create a single data frame based on combined series data

In [8]:
assessments = pd.DataFrame([x for x in df1['attributes']], dtype = 'object')
assessments = assessments.astype({'TaxYear': 'int64', 'ImprovementValue': 'int64','LandValue': 'int64','TotalValue': 'int64', 'ParcelNumber': 'str'})

## Create data frame from df that holds PIN and GPIN. Will be used to joined assessments, so that parcel area can be joined with assessments

In [9]:
df_key = pd.DataFrame(df[['PIN','GPIN']], dtype = 'str')

In [10]:
d = pd.merge(assessments, df_key, how='inner', left_on=['ParcelNumber'], right_on=['PIN'])

# Step 3: Analyze

In [11]:
#Create data frame of assessed values for all parcels by specified year
year_min = int(d['TaxYear'].min())
year_max = int(d['TaxYear'].max())
years = (x for x in range(year_min, year_max+1))
#total_val_col = d['TotalValue'][d['TaxYear'] == 2019].mean()
assessment_chart_list = []
for x in years:
    year_col = x
    land_col_mean = d['LandValue'][d['TaxYear'] == x].mean()
    imp_col_mean = d['ImprovementValue'][d['TaxYear'] == x].mean()
    total_col_mean = d['TotalValue'][d['TaxYear'] == x].mean()
    
    land_col_min = d['LandValue'][d['TaxYear'] == x].min()
    imp_col_min = d['ImprovementValue'][d['TaxYear'] == x].min()
    total_col_min = d['TotalValue'][d['TaxYear'] == x].min()
    
    land_col_max = d['LandValue'][d['TaxYear'] == x].max()
    imp_col_max = d['ImprovementValue'][d['TaxYear'] == x].max()
    total_col_max = d['TotalValue'][d['TaxYear'] == x].max()
    
    assessment_chart_list.append(
        [year_col, land_col_mean, imp_col_mean, total_col_mean, land_col_min, imp_col_min, 
         total_col_min, land_col_max, imp_col_max, total_col_max])
assessment_chart_cols = ['Tax Year', 'Mean Land Value', 'Mean Improvement Value', 'Mean Total Value',
    'Min Land Value', 'Min Improvement Value', 'Min Total Value', 'Max Land Value', 'Max Improvement Value', 
    'Max Total Value']
assessment_chart_df = pd.DataFrame(assessment_chart_list, columns = assessment_chart_cols)

In [12]:
plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Min Total Value'], linestyle = '--', color = '#000000', label='Min Total Value')
plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Mean Total Value'], linestyle = '-', color = '#000000', label='Mean Total Value')
plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Max Total Value'], linestyle = '-.', color = '#000000', label='Max Total Value')

plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Min Land Value'], linestyle = '--', color = '#41966d',  label='Min Land Value')
plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Mean Land Value'], linestyle = '-', color = '#41966d', label='Mean Land Value')
plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Max Land Value'],  linestyle = '-.', color = '#41966d', label='Max Land Value')

plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Min Improvement Value'], linestyle = '--', color = '#ff0000', label='Min Improvement Value')
plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Mean Improvement Value'], linestyle = '-', color = '#ff0000', label='Mean Improvement Value')
plt.plot(assessment_chart_df['Tax Year'], assessment_chart_df['Max Improvement Value'], linestyle = '-.', color = '#ff0000', label='Max Improvement Value')

plt.title('Assessed Value by Year')
plt.xlabel('Tax Years')
plt.ylabel('Assessment Value (10 Millions)')

plt.xticks(range(assessment_chart_df['Tax Year'].min(), assessment_chart_df['Tax Year'].max()+1), rotation=270)
plt.yticks(range(0, int(assessment_chart_df['Max Total Value'].max()), 5000000))

plt.legend(fontsize = 'small')
plt.savefig('plot.png', dpi=300, edgecolor = 'w', orientation='landscape')

In [13]:
def total_value_by_year(year):
    tv_df = d[['GPIN','TotalValue']][d['TaxYear'] == year]
    tv_df = tv_df.astype({'GPIN': 'int', 'TotalValue': 'int'})
    return tv_df

In [14]:
def choropleth_assessed_val_by_year(year):
    m = folium.Map(location=[38.03090, -78.48044], zoom_start=17, tiles='OpenStreetMap')
    folium.Choropleth(
        geo_data = parcel_url,
        data=total_value_by_year(year),
        columns=['GPIN', 'TotalValue'], 
        key_on='feature.properties.GPIN', 
        bins=[0, 250000, 500000, 1000000, 2000000, 5000000, 10000000, 16000000, 25000000, 50000000],
        fill_color= 'YlOrRd',
        nan_fill_color='black', 
        fill_opacity=0.9, 
        nan_fill_opacity=1, 
        line_color='black', 
        line_weight=1, 
        line_opacity=1, 
        name='Assessed Value',
        legend_name= f'Total Assessed Value {year}'
    ).add_to(m)
    return(m)

choropleth_assessed_val_by_year(2019)